In [3]:
import os     ## Allows operating system commands to be performed though Python
import sys    ## Allows you to exit a Python script for testing
import shutil ## Allows movement of files from one directory to another
import  glob  ## Global searching of strings as files
import string ## Fancy string manipulations
import pandas as pd

In [4]:
#change directory to folder with downloaded pdf's
os.chdir('C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Sun\\Ingredient_Disclosures')

In [5]:
pwd=os.getcwd()

In [6]:
#set exec path to folder with pdf-to-text application
execpath = 'C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\xpdfbin-win-3.04\\bin64\\'

In [7]:
#set names of applications 
htmlexec = "pdftohtml.exe"
txtexec = 'pdftotext.exe'

In [8]:
#set files to convert
files = glob.glob('C:\\Users\\akomandu\\PDF_Scrapes\\Sun\\Ingredient_Disclosures\\*.pdf')

In [9]:
#convert pdf files to text files
for f in files:
    filename = os.path.join("C:\\Users\\akomandu\\PDF_Scrapes\\Sun\\Ingredient_Disclosures", os.path.basename(f))
    dirname = os.path.basename(f)[:-4]
    
    txtcmd = os.path.join(execpath,txtexec)

    txtcmd = " ".join([txtcmd,"-table","-nopgbrk",f])

#     print(os.path.isfile(filename))
#    print(" ".join([txtcmd, filename]))
    
#     print(txtcmd)
    #print(os.path.isfile(os.path.join("C:\\Users\\akomandu\\PDF Scrapes\\Georgia Pacific MSDS",os.path.basename(filename))))
#     print(dirname)

    os.system(txtcmd)
#    os.system(" ".join([txtcmd, filename]))
    #textpath = os.path.join(pwd,"PDF Scrapes","Georgia Pacific MSDS", dirname  + " .txt ")
    
    #if (os.path.isfile(textpath)):
        #shutil.move(textpath,os.path.join(pwd,"text",dirname +".txt"))
    #else:
        ## Throw an error if the path doesn't exist
        #sys.exit("Path Not Found")


## Data Extraction

In [10]:
#create function for cleaning lines
clean = lambda dirty: ''.join(filter(string.printable.__contains__, dirty))

In [12]:
#set path to text files
pwd = ['C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Sun\\Ingredient_Disclosures']
os.chdir("".join(pwd))
pwd = os.getcwd()

In [13]:
#find all text files
text_files = glob.glob('*.txt')

In [102]:
from itertools import islice
import re

In [148]:
Product = []
Code = []
Ingredient = []
Function = []
File = []
for file in text_files:
    with open(file) as ifile:
        List1 = []
        List2 = []
        for line in ifile:
            cline = clean(line)
            cline = cline.lower()
            cline = cline.replace(',','_')
            cline = cline.replace(';','_')
            cline = cline.strip()
            cline = cline.split("  ")
            cline = [x.strip() for x in cline if x != ""]
            if 'friday_ 8:30am to 6:00pm est)' in cline:
                product = (list(islice(ifile,2))[1])
                #product = product.replace('                                ', ' ')
                product = re.sub(' +',' ',product)
                product = product[:-1]
            if 'upc code' in cline:
                code = (list(islice(ifile,2))[1])
                code = re.sub(' +',' ',code)
                code = code[:-1]
            if 'function' in cline:
                for line in ifile:
                    dline = clean(line)
                    dline = dline.lower()
                    dline = dline.replace(',','_')
                    dline = dline.replace(';','_')
                    dline = dline.strip()
                    dline = dline.split("  ")
                    dline = [x.strip() for x in dline if x != ""]
                    if len(dline) > 1:
                        List1.append(dline[0])
                        List2.append(dline[1])
                    else:
                        pass
                    if 'fragrance' in line:
                        break
        for i in List1:
            Ingredient.append(i)
            Product.append(product)
            Code.append(code)
            File.append(file)
        for i in List2:
            Function.append(i)

In [149]:
data = pd.DataFrame({
    'File':File,
    'Product':Product,
    'UPC_Code':Code,
    'Ingredient':Ingredient,
    'Function':Function
})

In [151]:
data.to_csv('Sun_INDS_07242018.csv')